In [1]:
from makiflow.save_recover import Builder
from makiflow.generators.ssd import LoadDataMethod
from makiflow.generators.ssd.gen_layers import InputGenLayerV2
from makiflow.generators.ssd.path_generators import RandomGeneratorSegment
import numpy as np
import makiflow as mf
mf.set_main_gpu(0)
from sklearn.utils import shuffle
from makiflow.metrics.od_metrics import mAP_maki_supported
import json
from glob import glob

In [2]:
map_method = LoadDataMethod(
    image_shape=[300, 300, 3],
    loc_shape=[11663, 4],
    loc_mask_shape=[11663],
    label_shape=[11663]
)

In [3]:

records = []
#fol = glob(f'/mnt/data/voc2012/tfr_sets/Mobilenetv2_1_data_rcnn_num_3/*')

#for i in range(len(fol)):
#    if 'test' not in fol[i]:
#        records += glob(fol[i] + '/*.tfrecord')

# 2012 75k
records += glob(f'/raid/rustam/obj_detection_tests/voc2012/tfr_sets/ResNet_sratch_det/origin/*.tfrecord') 

records += glob(f'/raid/rustam/obj_detection_tests/voc2012/tfr_sets/ResNet_sratch_det/flip_2/*.tfrecord')

records += glob(f'/raid/rustam/obj_detection_tests/voc2012/tfr_sets/ResNet_sratch_det/flip_0/*.tfrecord')

records += glob(f'/raid/rustam/obj_detection_tests/voc2012/tfr_sets/ResNet_sratch_det/flip_1_dark_blur/*.tfrecord')

# 2007 20k
records += glob(f'/raid/rustam/obj_detection_tests/voc2007/tfr_sets/ResNet_sratch_det/origin/*.tfrecord')

records += glob(f'/raid/rustam/obj_detection_tests/voc2007/tfr_sets/ResNet_sratch_det/flip_1_dark_blur/*.tfrecord')

records += glob(f'/raid/rustam/obj_detection_tests/voc2007/tfr_sets/ResNet_sratch_det/flip_1_3_brigth/*.tfrecord')

records += glob(f'/mnt/data/rustam/obj_detection_tests/voc2007/tfr_sets/ResNet_sratch_det/flip_1/*.tfrecord')

In [4]:
len(records)

257

In [5]:
gen = RandomGeneratorSegment(records)

In [6]:
batch_size = 64

In [7]:
#in_x = InputGenLayer(2, batch_size, records, name='input', map_operation=map_method, num_parallel_calls=5)
in_x = InputGenLayerV2(
    3,
    batch_size,
    gen,             
    name='input',
    map_operation=map_method, 
    num_parallel_calls=5,
    cycle_length=5,
    block_length=160,
    shuffle=True, 
    buffer_size=64*4,
)

AttributeError: module 'tensorflow.io' has no attribute 'FixedLenFeature'

In [ ]:
model = Builder.ssd_from_json('scratch_det/model.json', generator=in_x)

In [ ]:
in_x.get_iterator()

In [ ]:
import tensorflow as tf

In [ ]:
#model.set_huber_loss()

In [ ]:
model.set_generator(in_x)

In [ ]:
ses = tf.Session()

Create a change of learning rate

In [ ]:
global_step = tf.Variable(0, trainable=False)

one_epoch = int((len(records) * 320) / batch_size)

def get_piecewise_constant_decay():
    global global_step, one_epoch
    boundaries = [one_epoch * 100]
    boundaries += [boundaries[-1] + one_epoch * 25]
    values = [5e-2,5e-3, 5e-4]
    return tf.compat.v1.train.piecewise_constant(global_step, boundaries,
        values, name='supa_optimizer_lr')

def get_exp_decay(lr):
    global global_step, one_epoch
    return tf.compat.v1.train.exponential_decay(lr,
       global_step, one_epoch*2, 0.94, staircase=True, name='supa_optimizer_lr')

def get_cosine_decay(lr, steps=1,t_mul=2.0,m_mul=1.0,alpha=1e-4):
    global global_step, one_epoch
    return tf.train.cosine_decay_restarts(learning_rate=lr,
        global_step=global_step,
        first_decay_steps=one_epoch*steps,
        t_mul=t_mul,
        m_mul=m_mul,
        alpha=alpha,
        name='supa_optimizer_lr')

lr = get_piecewise_constant_decay()
ses.run(tf.initialize_variables([global_step]))

In [ ]:
model.set_session(ses)

In [ ]:
model.set_common_l2_weight_decay(1e-4)

Set up test data

In [ ]:
#from makiflow.models.ssd.tools import DataPreparator
from makiflow.models.ssd.tools.data_preparator_v2 import DataPreparatorV2 as DataPreparator

In [ ]:
with open('data/classes.json', 'r') as f:
    name2class = json.load(f)

with open('data/testset_2007.json', 'r') as f:
    d = json.load(f)

preparator = DataPreparator(d, name2class, '/mnt/data/rustam/obj_detection_tests/voc2007tests/VOCdevkit/VOC2007/JPEGImages/')
preparator.load_images()
preparator.resize_images_and_bboxes((300, 300))
imgs = preparator.get_images()

imgs = ((np.asarray(imgs) / 128) - 1).astype(np.float32)

true_boxes, true_labels = preparator.get_true_boxes_labels()

In [ ]:
len(imgs)

In [ ]:

def performe_test(model, name):
    preds = []
    for i in range(len(imgs) // batch_size):
        # preds = [confidences, locs]
        # `confidences` shape is [batch_sz, total_predictions, num_classes]
        # `locs` shape is [batch_sz, total_predictions, 4]
        preds += [model.predict(imgs[i*batch_size:(i+1)*batch_size])]
    
    p, r, ap, f1, unique_classes = mAP_maki_supported(preds, iou_threshold=0.5, conf_threshold=0.5, gboxes=true_boxes,glabels=true_labels)
    print('mAP is ',ap.mean())
    ap = np.round(ap,2)
    class2name = dict([(value,key) for key,value in name2class.items()])
    for i in range(len(ap)):
        print(class2name[i+1],'  ', ap[i])
    global_ap[name].append(ap.mean())

global_ap = {
    'focal': [],
    'maki': [],
    'scan': [],
    'topk': [],
    'quadratic': []
}

In [ ]:
loss_info_top_k = {
    'positive losses': [],
    'negative losses': [],
    'total losses': [],
    'loc losses': []
}

loss_info_maki = {
    'Total loss':[],
    'Maki Loss':[],
    'Loc loss':[],
}

loss_info_focal = {
    'total loss':[],
    'focal loss':[],
    'loc loss':[],
}

loss_info_square = {
    'Total loss':[],
    'Maki Loss':[],
    'Loc loss':[],
}

In [ ]:
epochs = 201
iterations = one_epoch
loc_loss_w = 1.0 # 1.5
neg_ratio = 3.0 # 3.0
gamma = 2
# 8e-4
#opt = tf.train.AdamOptimizer(learning_rate=lr)
opt = tf.train.MomentumOptimizer(lr,momentum=0.9,use_nesterov=True)
#opt = tf.train.RMSPropOptimizer(learning_rate=lr, momentum=0.9, decay=0.999)

In [ ]:
# MAKI

In [ ]:
for i in range(epochs):
    info = model.genfit_maki(optimizer=opt, loc_loss_weight=loc_loss_w, gamma=gamma,
                                  epochs=1, iterations=iterations, global_step=global_step)
    performe_test(model, 'maki')
    loss_info_maki['Total loss'].append(info['Total loss'][0])
    loss_info_maki['Maki Loss'].append(info['Maki Loss'][0])
    loss_info_maki['Loc loss'].append(info['Loc loss'][0])
    print('epoch is ', i)
    if i != 0 and i % 4000 == 0:
        model.save_weights(f'maki_{i}_zaet.ckpt')
        
#model.save_weights('top_k_map36.ckpt')
#reset()

In [ ]:
# FOCAL

In [ ]:
for i in range(epochs):
    info = model.genfit_focal(optimizer=opt, loc_loss_weight=loc_loss_w, gamma=gamma,
                                  epochs=1, iterations=iterations, global_step=global_step)
    performe_test(model, 'focal')
    loss_info_focal['total loss'].append(info['total loss'][0])
    loss_info_focal['focal loss'].append(info['focal loss'][0])
    loss_info_focal['loc loss'].append(info['loc loss'][0])
    print('epoch is ', i)
    if i != 0 and i % 4000 == 0:
        model.save_weights(f'focal_{i}_zaet.ckpt')
        
#model.save_weights('top_k_map36.ckpt')
#reset()

In [ ]:
# SQUEARE

In [ ]:
for i in range(epochs):
    info = model.genfit_quadratic_ce(optimizer=opt, loc_loss_weight=loc_loss_w,
                                  epochs=1, iterations=iterations, global_step=global_step)
    performe_test(model, 'quadratic')
    loss_info_square['Total loss'].append(info['Total loss'][0])
    loss_info_square['Maki Loss'].append(info['Maki Loss'][0])
    loss_info_square['Loc loss'].append(info['Loc loss'][0])
    print('epoch is ', i)
    if i != 0 and i % 4000 == 0:
        model.save_weights(f'square_{i}_zaet.ckpt')
        
#model.save_weights('top_k_map36.ckpt')
#reset()

In [ ]:
# TOP K

In [ ]:
for i in range(epochs):
    info = model.genfit_top_k(optimizer=opt, loc_loss_weight=loc_loss_w, neg_samples_ratio=neg_ratio,
                              epochs=1, iterations=iterations, global_step=global_step)
    performe_test(model, 'topk')
    loss_info_top_k['positive losses'].append(info['positive losses'][0])
    loss_info_top_k['negative losses'].append(info['negative losses'][0])
    loss_info_top_k['total losses'].append(info['total losses'][0])
    loss_info_top_k['loc losses'].append(info['loc losses'][0])
    print('epoch is ', i)
    if i != 0 and i % 4000 == 0:
        model.save_weights(f'top_k_{i}_zaet.ckpt')
    #if i != 0 and i % 10 == 0:
    #    loc_loss_w *= 0.92
    #if i != 0 and i % 10 == 0:
    #    neg_ratio *= 0.96
        
#model.save_weights('top_k_map36.ckpt')
#reset()

In [ ]:
2

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(global_ap['topk'])
plt.show()

In [ ]:
plt.plot(loss_info_top_k['positive losses'])
plt.show()

In [ ]:
plt.plot(loss_info_top_k['negative losses'])
plt.show()

In [ ]:
plt.plot(loss_info_top_k['total losses'])
plt.show()

In [ ]:
plt.plot(loss_info_top_k['loc losses'])
plt.show()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(global_ap['focal'])

In [ ]:
plt.plot(global_ap['maki'])

In [ ]:
plt.plot(global_ap['topk'])

In [ ]:
plt.plot(global_ap['scan'])

In [ ]:
plt.plot(global_ap['quadratic'])

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
from makiflow.models.ssd.ssd_utils import nms, draw_bounding_boxes
import matplotlib.pyplot as plt
def test_picture(images, conf_trashhold=0.6):
    global model
    global num2name

    classes, locs = model.predict(images)
    i = 0
    for class_pred, loc_pred, image in zip(classes, locs, images):
        pred = nms(loc_pred, class_pred, conf_trashhold=conf_trashhold, iou_trashhold=0.5)
        print(pred,' number is ',i)
        input_dict = {
          'bboxes': pred[0],
          'classes': [num2name[cl] for cl in pred[1]]
        }
        image_with_bb = draw_bounding_boxes(image, input_dict)
        fig = plt.figure()
        fig.set_size_inches(16, 9)

        axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])
        axes.imshow(image_with_bb)
        fig.savefig('results/'  +f'{i}.jpg')
        i += 1
        plt.close('all')

In [ ]:
import cv2 
import os
import numpy as np
arrs = []

path = '/mnt/data/voc2012/VOCdevkit/VOC2012/JPEGImages/'
img = None
taken = None
for elem in d['test_data']:
        path_image = path + elem['filename']
        testPic = cv2.imread(path_image)#NAME
        testPic = cv2.resize(testPic,(300,300))
        #testPic = cv2.cvtColor(testPic, cv2.COLOR_BGR2RGB)
        cal = (np.array(testPic) / 255).astype(np.float32)
        arrs.append(cal)
        img = testPic
        taken = elem
        break

In [ ]:
len(arrs)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img)

In [ ]:
taken

In [ ]:
test_picture(arrs[:32], conf_trashhold=0.6)

In [ ]:
num2name = dict([(value, name) for name, value in class2name.items()])

In [ ]:
num2name